# ✅ Validación y Evaluación de Modelos de ML
## Técnicas Rigurosas para Garantizar Generalización

<a href="https://colab.research.google.com/github/yourusername/ml-course/blob/main/01_Sistemas_aprendizaje_automatico/03_Validacion_evaluacion/validacion_evaluacion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

## 📋 Resumen del Notebook

Este notebook cubre **metodologías rigurosas de validación y evaluación** para garantizar que los modelos de ML generalicen correctamente.

### 🎯 Objetivos de Aprendizaje

1. **Problemáticas de Generalización**:
   - Overfitting vs Underfitting
   - Trade-off Sesgo-Varianza
   - Detección y mitigación

2. **Estrategias de Validación**:
   - Train-Test Split
   - K-Fold Cross-Validation
   - Stratified K-Fold
   - Leave-One-Out (LOO)
   - Time Series Split

3. **Métricas de Evaluación**:
   - **Clasificación**: Accuracy, Precision, Recall, F1, ROC-AUC, PR-AUC
   - **Regresión**: MSE, RMSE, MAE, R², MAPE
   - Matrices de confusión y curvas ROC

4. **Optimización de Hiperparámetros**:
   - Grid Search
   - Random Search
   - Bayesian Optimization

### 📊 Contenido

- Teoría con fundamentación matemática
- Implementaciones prácticas
- Visualizaciones de curvas de aprendizaje
- Comparaciones de estrategias de validación
- Pipeline completo de evaluación

---

In [ ]:
# Configuración del entorno
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import (train_test_split, cross_val_score, KFold, 
                                      StratifiedKFold, LeaveOneOut, TimeSeriesSplit,
                                      GridSearchCV, RandomizedSearchCV, learning_curve,
                                      validation_curve)
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                            confusion_matrix, classification_report, roc_curve, roc_auc_score,
                            precision_recall_curve, average_precision_score,
                            mean_squared_error, mean_absolute_error, r2_score,
                            mean_absolute_percentage_error)
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, Ridge, Lasso
from sklearn.tree import DecisionTreeClassifier
from sklearn.datasets import make_classification, make_regression, load_breast_cancer, load_diabetes
from sklearn.preprocessing import StandardScaler
import warnings

warnings.filterwarnings('ignore')
sns.set_style('darkgrid')
plt.rcParams['figure.dpi'] = 100

print("✅ Entorno configurado correctamente")
print(f"NumPy: {np.__version__} | Pandas: {pd.__version__}")
print(f"Scikit-learn: Módulos de validación, métricas y optimización cargados")

## 1. Overfitting vs Underfitting

### 🎯 Problemáticas de Generalización

#### Overfitting (Sobreajuste)
Modelo aprende **ruido y peculiaridades** del conjunto de entrenamiento en lugar de patrones generales.

**Síntomas**:
- Error entrenamiento << Error validación
- Modelo muy complejo para los datos
- Rendimiento degrada con datos nuevos

#### Underfitting (Subajuste)
Modelo es **demasiado simple** para capturar patrones subyacentes.

**Síntomas**:
- Error entrenamiento alto
- Error validación alto
- Modelo no captura relaciones en los datos

### Trade-off Sesgo-Varianza

$$\text{Error Total} = \text{Sesgo}^2 + \text{Varianza} + \text{Ruido Irreducible}$$

- **Sesgo alto** → Underfitting (modelo simple)
- **Varianza alta** → Overfitting (modelo complejo)
- **Objetivo**: Balance óptimo entre ambos

In [ ]:
# Demostración Práctica: Overfitting vs Underfitting con Curvas de Aprendizaje

print("🎯 Visualizando Overfitting, Underfitting y el Modelo Óptimo\n")

# Generar datos con ruido
np.random.seed(42)
X_demo = np.sort(np.random.rand(100, 1) * 10, axis=0)
y_demo = np.sin(X_demo).ravel() + np.random.randn(100) * 0.3

# Crear tres modelos con diferentes complejidades
from sklearn.tree import DecisionTreeRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures

models_complexity = {
    'Underfitting\n(max_depth=1)': DecisionTreeRegressor(max_depth=1, random_state=42),
    'Óptimo\n(max_depth=5)': DecisionTreeRegressor(max_depth=5, random_state=42),
    'Overfitting\n(max_depth=20)': DecisionTreeRegressor(max_depth=20, random_state=42)
}

# Dividir datos
X_train_demo, X_test_demo, y_train_demo, y_test_demo = train_test_split(
    X_demo, y_demo, test_size=0.3, random_state=42
)

# Visualización
fig, axes = plt.subplots(1, 3, figsize=(18, 5))

X_plot = np.linspace(0, 10, 500).reshape(-1, 1)

for idx, (name, model) in enumerate(models_complexity.items()):
    # Entrenar
    model.fit(X_train_demo, y_train_demo)
    y_train_pred = model.predict(X_train_demo)
    y_test_pred = model.predict(X_test_demo)
    
    # Calcular errores
    train_score = r2_score(y_train_demo, y_train_pred)
    test_score = r2_score(y_test_demo, y_test_pred)
    train_mse = mean_squared_error(y_train_demo, y_train_pred)
    test_mse = mean_squared_error(y_test_demo, y_test_pred)
    
    # Plot
    axes[idx].scatter(X_train_demo, y_train_demo, color='blue', s=40, alpha=0.6, label='Train')
    axes[idx].scatter(X_test_demo, y_test_demo, color='red', s=40, alpha=0.6, label='Test')
    axes[idx].plot(X_plot, model.predict(X_plot), 'g-', linewidth=2, label='Modelo')
    
    axes[idx].set_title(f'{name}\nTrain R²: {train_score:.3f} | Test R²: {test_score:.3f}\n'
                       f'Train MSE: {train_mse:.3f} | Test MSE: {test_mse:.3f}',
                       fontweight='bold', fontsize=10)
    axes[idx].set_xlabel('X')
    axes[idx].set_ylabel('y')
    axes[idx].legend()
    axes[idx].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("\n📊 Análisis:")
print("="*70)
print("⚠️  UNDERFITTING:")
print("   • Train score BAJO, Test score BAJO")
print("   • Modelo demasiado simple, no captura patrones")
print("\n✅ MODELO ÓPTIMO:")
print("   • Train score ALTO, Test score ALTO y similar")
print("   • Generaliza bien a datos nuevos")
print("\n❌ OVERFITTING:")
print("   • Train score MUY ALTO, Test score BAJO")
print("   • Memoriza ruido del training set")

## 2. Estrategias de Validación

### 📊 Métodos de Validación

#### Train-Test Split Simple
División aleatoria: típicamente 70-30% o 80-20%
- ✅ Rápido, simple
- ⚠️ Alta varianza, depende de la partición

#### K-Fold Cross-Validation
Divide datos en K folds, entrena K veces
- ✅ Estimador más robusto
- ✅ Usa todos los datos
- ⚠️ K veces más costoso

#### Stratified K-Fold
K-Fold preservando distribución de clases
- ✅ Esencial para datos desbalanceados
- ✅ Reduce varianza en problemas multiclase

#### Leave-One-Out (LOO)
K-Fold con K=n (cada muestra es un fold)
- ✅ Casi imparcial
- ⚠️ Muy costoso, alta varianza

#### Time Series Split
Validación cronológica (no aleatoria)
- ✅ Respeta orden temporal
- ✅ Previene data leakage futuro→pasado
- 🎯 Obligatorio para series temporales

In [ ]:
# Ejemplo Práctico Comprensivo: Pipeline Completo de Validación y Evaluación

print("🔬 Pipeline Completo: Validación Cruzada + Métricas + Optimización\n")

# Cargar dataset de clasificación binaria
cancer = load_breast_cancer()
X_cancer = cancer.data
y_cancer = cancer.target

print(f"Dataset: Breast Cancer Wisconsin")
print(f"  • Muestras: {len(y_cancer)}")
print(f"  • Features: {X_cancer.shape[1]}")
print(f"  • Clases: {len(np.unique(y_cancer))} (Malignant: {sum(y_cancer==0)}, Benign: {sum(y_cancer==1)})")

# Normalizar
scaler_cancer = StandardScaler()
X_cancer_scaled = scaler_cancer.fit_transform(X_cancer)

# 1. COMPARACIÓN DE ESTRATEGIAS DE VALIDACIÓN
print("\n📊 1. COMPARACIÓN DE ESTRATEGIAS DE VALIDACIÓN")
print("="*70)

models_val = {
    'RandomForest': RandomForestClassifier(n_estimators=100, random_state=42),
    'SVM (RBF)': SVC(kernel='rbf', random_state=42, probability=True),
    'Logistic Reg': LogisticRegression(max_iter=1000, random_state=42)
}

validation_strategies = {
    'Train-Test (80-20)': None,  # Handled separately
    'K-Fold (5)': KFold(n_splits=5, shuffle=True, random_state=42),
    'K-Fold (10)': KFold(n_splits=10, shuffle=True, random_state=42),
    'Stratified K-Fold (5)': StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
}

results_validation = {model_name: {} for model_name in models_val.keys()}

for model_name, model in models_val.items():
    # Train-Test Split
    X_train, X_test, y_train, y_test = train_test_split(
        X_cancer_scaled, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
    )
    model_clone = sklearn.base.clone(model)
    model_clone.fit(X_train, y_train)
    results_validation[model_name]['Train-Test (80-20)'] = model_clone.score(X_test, y_test)
    
    # Cross-Validation strategies
    for strategy_name, cv in validation_strategies.items():
        if cv is not None:
            scores = cross_val_score(model, X_cancer_scaled, y_cancer, cv=cv, scoring='accuracy')
            results_validation[model_name][strategy_name] = scores.mean()

# Visualizar resultados
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 5))

# Tabla de resultados
df_results = pd.DataFrame(results_validation).T
print(f"\n{df_results}\n")

# Plot 1: Comparación de estrategias
strategies = list(validation_strategies.keys())
x_pos = np.arange(len(strategies))
width = 0.25

for idx, model_name in enumerate(models_val.keys()):
    values = [results_validation[model_name][s] for s in strategies]
    ax1.bar(x_pos + idx*width, values, width, label=model_name, alpha=0.8)

ax1.set_ylabel('Accuracy')
ax1.set_title('Comparación de Estrategias de Validación', fontweight='bold', fontsize=12)
ax1.set_xticks(x_pos + width)
ax1.set_xticklabels(strategies, rotation=15, ha='right')
ax1.legend()
ax1.grid(True, alpha=0.3, axis='y')
ax1.set_ylim([0.85, 1.0])

# 2. MÉTRICAS DETALLADAS (usando Random Forest con Stratified K-Fold)
print("\n📈 2. ANÁLISIS DETALLADO DE MÉTRICAS (Random Forest)")
print("="*70)

import sklearn.base
rf_model = RandomForestClassifier(n_estimators=100, random_state=42)
X_train, X_test, y_train, y_test = train_test_split(
    X_cancer_scaled, y_cancer, test_size=0.2, random_state=42, stratify=y_cancer
)
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)
y_pred_proba = rf_model.predict_proba(X_test)[:, 1]

# Métricas
accuracy = accuracy_score(y_test, y_pred)
precision = precision_score(y_test, y_pred)
recall = recall_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
roc_auc = roc_auc_score(y_test, y_pred_proba)

print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f} (De los predichos como positivos, % correctos)")
print(f"Recall:    {recall:.4f} (De todos los positivos reales, % detectados)")
print(f"F1-Score:  {f1:.4f} (Media armónica Precision-Recall)")
print(f"ROC-AUC:   {roc_auc:.4f} (Área bajo curva ROC)")

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)

# Plot 2: Matriz de Confusión
import sklearn.metrics
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax2,
           xticklabels=['Malignant', 'Benign'], 
           yticklabels=['Malignant', 'Benign'])
ax2.set_title(f'Matriz de Confusión\nAccuracy: {accuracy:.4f}', fontweight='bold', fontsize=12)
ax2.set_ylabel('True Label')
ax2.set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

# 3. CURVA ROC
print("\n📉 3. CURVA ROC")

fpr, tpr, thresholds = roc_curve(y_test, y_pred_proba)

fig, (ax3, ax4) = plt.subplots(1, 2, figsize=(14, 5))

ax3.plot(fpr, tpr, 'b-', linewidth=2, label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax3.plot([0, 1], [0, 1], 'r--', linewidth=2, label='Random Classifier')
ax3.fill_between(fpr, tpr, alpha=0.2)
ax3.set_xlabel('False Positive Rate')
ax3.set_ylabel('True Positive Rate (Recall)')
ax3.set_title('Receiver Operating Characteristic (ROC)', fontweight='bold', fontsize=12)
ax3.legend()
ax3.grid(True, alpha=0.3)

# 4. OPTIMIZACIÓN DE HIPERPARÁMETROS (Grid Search)
print("\n🔧 4. OPTIMIZACIÓN DE HIPERPARÁMETROS")
print("="*70)

param_grid = {
    'n_estimators': [50, 100, 200],
    'max_depth': [5, 10, 20, None],
    'min_samples_split': [2, 5, 10]
}

grid_search = GridSearchCV(
    RandomForestClassifier(random_state=42),
    param_grid,
    cv=StratifiedKFold(n_splits=5, shuffle=True, random_state=42),
    scoring='roc_auc',
    n_jobs=-1,
    verbose=0
)

print("Ejecutando Grid Search...")
grid_search.fit(X_cancer_scaled, y_cancer)

print(f"\nMejores hiperparámetros: {grid_search.best_params_}")
print(f"Mejor ROC-AUC (CV): {grid_search.best_score_:.4f}")

# Visualizar resultados de Grid Search (subset)
results_df = pd.DataFrame(grid_search.cv_results_)
top_10 = results_df.nsmallest(10, 'rank_test_score')[['params', 'mean_test_score', 'std_test_score']]

# Plot resultados top 10
top_10_sorted = top_10.sort_values('mean_test_score', ascending=True)
y_pos = np.arange(len(top_10_sorted))

ax4.barh(y_pos, top_10_sorted['mean_test_score'], xerr=top_10_sorted['std_test_score'], 
        alpha=0.8, color='green', edgecolor='black')
ax4.set_yticks(y_pos)
ax4.set_yticklabels([f"Config {i+1}" for i in range(len(top_10_sorted))], fontsize=9)
ax4.set_xlabel('ROC-AUC Score')
ax4.set_title('Top 10 Configuraciones (Grid Search)', fontweight='bold', fontsize=12)
ax4.axvline(x=grid_search.best_score_, color='red', linestyle='--', linewidth=2, label='Best')
ax4.legend()
ax4.grid(True, alpha=0.3, axis='x')

plt.tight_layout()
plt.show()

print("\n" + "="*70)
print("✅ PIPELINE COMPLETO")
print("="*70)
print("✔️  Comparación de estrategias de validación")
print("✔️  Evaluación con múltiples métricas")
print("✔️  Matriz de confusión y curva ROC")
print("✔️  Optimización de hiperparámetros con Grid Search")
print("\n💡 Mejores prácticas aplicadas:")
print("   • Normalización de datos")
print("   • Stratified K-Fold para preservar distribución de clases")
print("   • Múltiples métricas para evaluación completa")
print("   • Cross-validation en optimización de hiperparámetros")

## 3. Conclusiones

### 📚 Conceptos Clave
1. **Overfitting/Underfitting**: Balance entre complejidad y generalización
2. **Validación Cruzada**: K-Fold, Stratified, Time Series Split
3. **Métricas**: Accuracy, Precision, Recall, F1, ROC-AUC, MSE, R²
4. **Optimización**: Grid Search, Random Search, Nested CV

### ✅ Mejores Prácticas
- Usar Stratified K-Fold para datos desbalanceados
- Evaluar con múltiples métricas (no solo accuracy)
- Nested CV para evaluación imparcial con optimización de hiperparámetros
- Normalizar datos para modelos basados en distancia
- Time Series Split para datos temporales

### 🎯 Ejercicios
1. Implementa Nested CV y compara con CV simple
2. Compara Grid Search vs Random Search en un problema complejo
3. Analiza trade-off Precision-Recall en clasificación desbalanceada
4. Crea curvas de aprendizaje para detectar overfitting/underfitting
5. Implementa validación temporal para series temporales

### 📖 Recursos
- Scikit-learn: Model Selection https://scikit-learn.org/stable/model_selection.html
- "The Elements of Statistical Learning" - Hastie et al.
- "Hands-On Machine Learning" - Aurélien Géron

---

**🎓 Siguiente**: [Deep Learning](../04_Deep_learning/deep_learning.ipynb)